# Darts

In [6]:
pip install darts

     -------------------------------------- 589.7/589.7 kB 2.9 MB/s eta 0:00:00
     ---------------------------------------- 73.9/73.9 MB 2.9 MB/s eta 0:00:00
     -------------------------------------- 435.6/435.6 kB 1.6 MB/s eta 0:00:00
     -------------------------------------- 800.3/800.3 kB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 43.8/43.8 kB ? eta 0:00:00
     -------------------------------------- 147.7/147.7 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 89.1/89.1 MB 3.7 MB/s eta 0:00:00
     -------------------------------------- 162.6/162.6 MB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 92.0/92.0 kB 2.6 MB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 3.6 MB/s eta 0:00:00
     -------------------------------------- 970.0/970.0 kB 4.1 MB/s eta 0:00:00
     ----------------------

In [5]:
import pandas as pd
from darts import TimeSeries

In [6]:
# import csv dataset
df=pd.read_csv('divvy-tripdata_cleaned.csv',delimiter=",")
df=pd.DataFrame(df)
df=df.dropna()
df=df.sort_values("start_time")

In [7]:
# series = TimeSeries.from_dataframe(df, "start_time", "trip_duration")
df

,trip_id,start_time,stop_time,bike_id,trip_duration,from_station_id,from_station_name,to_station_id,to_station_name,user_type,...,from_longitude,from_location,to_latitude,to_longitude,to_location,date,year,month,day,day_of_week
21809,1109806,01/01/2014 03:57:00 PM,01/01/2014 04:05:00 PM,1920,479,73,Jefferson St & Monroe St,134,Peoria St & Jackson Blvd,Subscriber,...,-87.642746,POINT (-87.642746 41.880422),41.877749,-87.649633,POINT (-87.649633 41.877749),2014-01-01,2014,1,1,Wednesday
1861,1109849,01/01/2014 05:10:00 PM,01/01/2014 05:18:00 PM,2931,499,123,California Ave & Milwaukee Ave,290,Kedzie Ave & Palmer Ct,Subscriber,...,-87.697153,POINT (-87.697153 41.922695),41.921525,-87.707322,POINT (-87.707322 41.921525),2014-01-01,2014,1,1,Wednesday
2157,1109932,01/01/2014 07:53:00 PM,01/01/2014 07:57:00 PM,1945,258,176,Clark St & Elm St,176,Clark St & Elm St,Subscriber,...,-87.631253,POINT (-87.631253 41.903233),41.903233,-87.631253,POINT (-87.631253 41.903233),2014-01-01,2014,1,1,Wednesday
49965,1109940,01/01/2014 08:14:00 PM,01/01/2014 08:29:00 PM,225,884,324,Stockton Dr & Wrightwood Ave,324,Stockton Dr & Wrightwood Ave,Subscriber,...,-87.638742,POINT (-87.638742 41.93132),41.931320,-87.638742,POINT (-87.638742 41.93132),2014-01-01,2014,1,1,Wednesday
25884,1109983,01/02/2014 05:04:00 AM,01/02/2014 05:13:00 AM,1655,523,303,Broadway & Cornelia Ave,117,Wilton Ave & Belmont Ave,Subscriber,...,-87.645980,POINT (-87.64598 41.945512),41.940180,-87.653040,POINT (-87.65304 41.94018),2014-01-02,2014,1,2,Thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31352,1108260,12/31/2013 08:32:00 AM,12/31/2013 08:40:00 AM,997,429,289,Wells St & Concord Ln,141,Clark St & Lincoln Ave,Subscriber,...,-87.634656,POINT (-87.634656 41.912133),41.915689,-87.634600,POINT (-87.6346 41.915689),2013-12-31,2013,12,31,Tuesday
38232,1108302,12/31/2013 08:58:00 AM,12/31/2013 09:12:00 AM,1419,845,56,Desplaines St & Kinzie St,26,McClurg Ct & Illinois St,Subscriber,...,-87.644448,POINT (-87.644448 41.888716),41.891020,-87.617300,POINT (-87.6173 41.89102),2013-12-31,2013,12,31,Tuesday
63903,1108328,12/31/2013 09:33:00 AM,12/31/2013 10:04:00 AM,2989,1845,220,Hampden Ct & Diversey Pkwy,173,Mies van der Rohe Way & Chicago Ave,Subscriber,...,-87.642385,POINT (-87.642385 41.93262),41.896910,-87.621743,POINT (-87.621743 41.89691),2013-12-31,2013,12,31,Tuesday
74949,1108498,12/31/2013 12:04:00 PM,12/31/2013 12:08:00 PM,2482,228,346,Ada St & Washington Blvd,80,Aberdeen St & Madison St,Subscriber,...,-87.661206,POINT (-87.661206 41.88283),41.880456,-87.653929,POINT (-87.653929 41.880456),2013-12-31,2013,12,31,Tuesday


In [9]:
df=df.sort_values("date")

In [11]:
series = TimeSeries.from_dataframe(df, "date", "trip_duration", fill_missing_dates=True, freq=None)

ValueError: Could not observe an inferred frequency. An explicit frequency must be evident over a span of at least 3 consecutive time stamps in the input data. For more information about frequency aliases, read https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases


ValueError: Could not observe an inferred frequency. An explicit frequency must be evident over a span of at least 3 consecutive time stamps in the input data. For more information about frequency aliases, read https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases